# Model creation

In this example, we go through the process of generating a `FEModel` which can then be used for further computations.

## Model initialization

First, we initialize the `FEModel`.

In [1]:
from pathlib import Path
from shamo.model import FEModel

model = FEModel("example_model", str(Path("./derivatives")))

After that, a directory named `example_model` appears in the current directory.

## Mesh generation

First, the model must have a mesh to store geometric information. The `FEModel` class provides four methods to generate a mesh depending on the data you have at the beginning:
- `model.fem_from_labels()` is used if the input data is a `np.ndarray` containing multiple integer labels from `0` (void) to `n` (n-th tissue).
- `model.fem_from_nii()` is used if the input data is a `.nii` image containing the exact same data as the `np.ndarray` described above.
- `model.fem_from_masks()` is used if the input data is a series of `np.ndarray` containing each separate tissue binary masks.
- `model.fem_from_niis()` is used if the input data is a series of `.nii` images containing the data described above.

For the purpose of this example, we generate a multilabel `np.array` from scratch and pass it to `model.fem_from_labels()` but all the methods described above give the same result.

In [2]:
import numpy as np
from shamo.model import MeshConfig

# Generate the parameters
labels = np.ones((11, 11, 11), dtype=np.uint8)
labels[3:-3, 3:-3, 3:-3] = 2
labels[2:-5, 2:-5, 2:-5] = 3
labels[5, :, :4] = 0

tissues = ["a", "b", "c"]

affine = np.diag([1, 1, 1, 1])

mesh_config = MeshConfig(facet_distance=1, cell_size=0.5)

# Generate the mesh
model.fem_from_labels(labels, tissues, affine, mesh_config=mesh_config)
model.save()

After this, a file named `example_model.msh` is created under the previously created `example_model/` directory. You can open it with [Gmsh](http://gmsh.info/). The `FEModel` also has a new attribute `tissues`. Each key-value pair represents a tissue of the model determined by a `Tissue` object.

## Sensors placement

Now that we have a proper mesh, it is time to add the sensors. To do so, we can use the two following methods:
- `model.add_sensor_on_tissue()` to add a single sensor.
- `model.add_sensors_on_tissues()` to add multiple sensors at once.

In this example, we define 8 sensors located. For each, we set a name and a location.

In [3]:
# Define the sensors
coordinates = [(x, y, z) for x in (0, 10) for y in (0, 10) for z in (0, 10)]
sensors = {chr(ord("A") + i): c for i, c in enumerate(coordinates)}

# Add the sensors
model.add_sensors_on_tissue(sensors, "a")

/home/martin/Bureau/codes/shamo/.venv/lib/python3.7/site-packages/numpy/ctypeslib.py:523: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


{'mesh_path': 'example_model.msh',
 'tissues': {'a': {'volume_group': 1,
   'volume_entity': [1],
   'surface_group': 2,
   'surface_entity': [1]},
  'b': {'volume_group': 3,
   'volume_entity': [2],
   'surface_group': 4,
   'surface_entity': [2]},
  'c': {'volume_group': 5,
   'volume_entity': [3],
   'surface_group': 6,
   'surface_entity': [3]}},
 'sensors': {'A': {'real_coordinates': (0, 0, 0),
   'mesh_coordinates': (0.4722380936145782,
    0.06859377026557922,
    0.002851892728358507),
   'group': 1,
   'entity': 1,
   'on_tissue': 'a'},
  'B': {'real_coordinates': (0, 0, 10),
   'mesh_coordinates': (0.4705657660961151,
    0.001449603820219636,
    9.708480834960938),
   'group': 2,
   'entity': 2,
   'on_tissue': 'a'},
  'C': {'real_coordinates': (0, 10, 0),
   'mesh_coordinates': (0.3559772670269012,
    9.781022071838379,
    0.001235880539752543),
   'group': 3,
   'entity': 3,
   'on_tissue': 'a'},
  'D': {'real_coordinates': (0, 10, 10),
   'mesh_coordinates': (-0.000983

Now, the mesh contains multiple sensors. To know the error of placement of each of them, simply call `Sensor.coordinates_error`:

In [4]:
for name, sensor in model.sensors.items():
    print("{}: {:0.3f}[mm]".format(name, sensor.coordinates_error))

A: 0.477[mm]
B: 0.554[mm]
C: 0.418[mm]
D: 0.520[mm]
E: 0.351[mm]
F: 0.456[mm]
G: 0.098[mm]
H: 0.445[mm]
